In [1]:
# importing libraries
import numpy as np
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup

In [10]:
base_url = "https://www.worldfootball.net/schedule/eng-premier-league-2018-2019-spieltag/"

In [11]:
# Here we are creating two different dataframes. One with weekly standing and one with weekly points for each team

team1 = [list(range(1,21))]
team2 = [list(range(1,21))]
colnames1 = ['Position']
colnames2 = ['Position']

# for each matchweek we need to scrape a page
for matchweek in range(1, 39):
    standing = []
    points = []
    colnames1.append("Team"+str(matchweek))
    colnames1.append("Points"+str(matchweek))
    # concantenate strings to create url
    url = base_url + str(matchweek)
    
    # scrape a webpage
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # select all tables from page (there are 2 tables on page)
    data = soup.select('table.standard_tabelle')
    
    # select all 'a' tags (Team Names) from second table (point table)
    for team in data[1].findAll('a'):
        standing.append(team.text)
    
    # process one row at a time in table to get points 
    for team in data[1].findAll('tr'):
        count = 0
        # process each record in row at a time
        for rec in team.findAll('td'):
            # only interested in 10th record (points)
            if (count == 9):
                points.append(rec.text)
            count += 1
        
    team1.append(standing)
    team1.append(points)

# creating data frame for weekly team standing
weekly_standing = pd.DataFrame.from_records(team1).T
weekly_standing.columns = colnames1

# creating data frame for weekly points
#weekly_points = pd.DataFrame.from_records(team2).T
#weekly_points.columns = colnames2

In [16]:
weekly_standing.head()

,Position,Team1,Points1,Team2,Points2,Team3,Points3,Team4,Points4,Team5,...,Team34,Points34,Team35,Points35,Team36,Points36,Team37,Points37,Team38,Points38
0,1,Liverpool FC,3,Manchester City,6,Liverpool FC,9,Liverpool FC,12,Chelsea FC,...,Manchester City,86,Manchester City,89,Manchester City,92,Manchester City,95,Manchester City,98
1,2,Chelsea FC,3,Liverpool FC,6,Tottenham Hotspur,9,Chelsea FC,12,Liverpool FC,...,Liverpool FC,85,Liverpool FC,88,Liverpool FC,91,Liverpool FC,94,Liverpool FC,97
2,3,AFC Bournemouth,3,Chelsea FC,6,Chelsea FC,9,Watford FC,12,Manchester City,...,Tottenham Hotspur,70,Tottenham Hotspur,70,Tottenham Hotspur,70,Chelsea FC,71,Chelsea FC,72
3,4,Crystal Palace,3,Watford FC,6,Watford FC,9,Manchester City,10,Watford FC,...,Arsenal FC,66,Chelsea FC,67,Chelsea FC,68,Tottenham Hotspur,70,Tottenham Hotspur,71
4,5,Manchester City,3,Tottenham Hotspur,6,Manchester City,7,Tottenham Hotspur,9,AFC Bournemouth,...,Chelsea FC,66,Arsenal FC,66,Arsenal FC,66,Arsenal FC,67,Arsenal FC,70


In [12]:
weekly_points.head()

NameError: name 'weekly_points' is not defined

In [17]:
# exporting these two files in csv format
weekly_standing.to_csv(path_or_buf= "weekly_standing.csv",index=False)
#weekly_points.to_csv(path_or_buf= "weekly_points.csv",index=False)

In [5]:
def parse_page(link):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [ ]:
def get_team_names(tables):
    for team in data[1].findAll('a'):
        standing.append(team.text)
    return standing

In [ ]:
def get_points(tables):
    names = get_team_names(tables)
    
    for team in data[1].findAll('tr'):
        count = 0
        # process each record in row at a time
        for rec in team.findAll('td'):
            # only interested in 10th record (points)
            if (count == 9):
                points.append(rec.text)
            count += 1
            
    return names.append(points)